In [192]:
import numpy as np
import pandas as pd
import math

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

# Import Data

In [193]:
# Import training and holdout dataframes
train_df = pd.read_csv('kc_house_data_train.csv')
hold_df = pd.read_csv('kc_house_data_test_features.csv')

# Create Features

In [194]:
# Print column names
print(train_df.keys())

Index(['Unnamed: 0', 'id', 'date', 'price', 'bedrooms', 'bathrooms',
       'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
       'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [195]:
# Function to create house ages
def set_ages(df):
    df['age'] = 2015 - df['yr_built']
    return df

# Create house ages for dataframes
# train_df = set_age(train_df)
# hold_df = set_age(hold_df)

In [196]:
# Function to transform year renovated
def trans_ren(df):
    df['yr_renovated'] = df['yr_renovated'] - 1933
    return df

# Transorm year renovated for dataframes
# train_df = trans_ren(train_df)
# hold_df = trans_ren(hold_df)

In [ ]:
# Function for creating grade dummy variables
def set_grades(df):
    dummies = pd.get_dummies(df['grade'], prefix='grade', drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    return df

# Create grade dummy variables for dataframes
train_df = set_grades(train_df)
hold_df = set_grades(hold_df)


In [197]:
# Function for creating year dummy variables
def set_years(df):
    df['year'] = df['date'].apply(lambda x: int(x[0:4]))
    dummies = pd.get_dummies(df['year'], drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    for year in dummies.keys():
        df[year] = df[year] * df['sqft_living']
    df.drop('year', axis=1, inplace=True)
    return df

# Create year dummy variables for dataframes
# train_df = set_years(train_df)
# hold_df = set_years(hold_df)

In [198]:
# Function for creating month dummy variables
def set_months(df):
#     months = {1:'jan', 2:'feb', 3:'mar', 4:'apr', 5:'may', 6:'jun',
#               7:'jul', 8:'aug', 9:'sep', 10:'oct', 11:'nov', 12:'dec'}
    df['month'] = df['date'].apply(lambda x: int(x[4:6]))
    dummies = pd.get_dummies(df['month'], drop_first=True)
#     for key in dummies.keys():
#         if key in months.keys():
#             dummies.rename({key : months[key]}, axis=1, inplace=True)
    df = pd.concat([df, dummies], axis=1)
    for month in dummies.keys():
        df[month] = df[month] * df['sqft_living']
    df.drop('month', axis=1, inplace=True)
    return df

# Create month dummy variables for dataframes
# train_df = set_months(train_df)
# hold_df = set_months(hold_df)

In [199]:
# Function for creating zipcode dummy variables
def set_zipcodes(df):
    dummies = pd.get_dummies(df['zipcode'], drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    for zipcode in dummies.keys():
        df[zipcode] = df[zipcode] * df['sqft_living']
    return df

# Create zipcode dummy variables for dataframes
# train_df = set_zipcodes(train_df)
# hold_df = set_zipcodes(hold_df)

In [200]:
# Function to transform waterfront variable
def trans_waterfront(df):
    df['waterfront'] = df['waterfront'] * df['sqft_living']
    return df

# Transform waterfront variable for dataframes
# train_df = trans_waterfront(train_df)
# hold_df = trans_waterfront(hold_df)

# Apply Features to Dataframes

In [201]:
# Function to create all features for dataframes
def create_featuers(df):
    df = set_ages(df)
    df = trans_ren(df)
    df = set_years(df)
    df = set_months(df)
    df = set_zipcodes(df)
    df = trans_waterfront(df)
    return df

# Create all features for dataframes
train_df = create_featuers(train_df)
hold_df = create_featuers(hold_df)

# Fix Outliers

In [202]:
# Function to transform extreme values
def fix_outliers(df, cols):
    for col in cols:
        if df[col].eq(0).any():
            filt = df[col] == 0
            df[f'{col}_nan'] = np.where(filt, np.nan, df[col])
            std = df[f'{col}_nan'].std()
            mean = df[f'{col}_nan'].mean()
            del df[f'{col}_nan']
        else:
            std = df[col].std()
            mean = df[col].mean()
        value = mean+(8*std)
        df[col] = df[col].apply(lambda x: value if (x>value) else x)
    return df

# Transform extreme values for specified columns dataframes
outlier_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above',
                'sqft_basement', 'sqft_living15', 'sqft_lot15']

train_df = fix_outliers(train_df, outlier_cols)
hold_df = fix_outliers(hold_df, outlier_cols)

# EDA

In [203]:
# # Function to create feature pplots
# def pplot_features(df, features):
#     pp_rows = [features[i:i+4] for i in range(0, len(features), 4)]
#     for row in pp_rows:
#         pp = sns.pairplot(data=df, y_vars=['price'], x_vars=row, kind='reg', height=3)
#     return None

# # Create feature pplots for training df
# pplot_features(train_df, features)

# Split Data Into Train, Test Data

In [204]:
def tt_split(df):
    ignore = ['Unnamed: 0', 'id', 'price', 'date', 'yr_built', 'zipcode',
          'lat', 'long', 'sqft_living', 'sqft_above']
    Y = df['price']
    for col in ignore:
        df = df.drop(columns=col, axis=1)
    x_train, x_test, y_train, y_test = train_test_split(df, Y, random_state=22,test_size=0.2)
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = tt_split(train_df)


In [205]:
x_hold = hold_df.drop(columns=['Unnamed: 0', 'id', 'date', 'yr_built', 'zipcode',
          'lat', 'long', 'sqft_living', 'sqft_above'])

# Run Model

In [206]:
model = sm.OLS(y_train, sm.add_constant(x_train[[key for key in x_train]])).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.880
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     1070.
Date:                Mon, 04 May 2020   Prob (F-statistic):               0.00
Time:                        11:16:32   Log-Likelihood:            -1.8250e+05
No. Observations:               13832   AIC:                         3.652e+05
Df Residuals:                   13737   BIC:                         3.659e+05
Df Model:                          94                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -2.653e+05   1.58e+04    -16.812      0.000   -2.96e+05   -2.34e+05
bedrooms      -2.144e+04   1571.133    -13.649      0.000   -2.45e+04   -1.84e+04
bathrooms       2.14e+04   2649.855      8.075      0.000    1.62e+04    2.66e+04
sqft_lot          0.6310      0.057     11.001      0.000       0.519       0.743
floors        -4.358e+04   3136.629    -13.895      0.000   -4.97e+04   -3.74e+04
waterfront      235.8311      3.739     63.071      0.000     228.502     243.160
view           4.716e+04   1763.670     26.738      0.000    4.37e+04    5.06e+04
condition      2.289e+04   1926.577     11.883      0.000    1.91e+04    2.67e+04
grade          5.088e+04   1814.169     28.046      0.000    4.73e+04    5.44e+04
sqft_basement   -87.6328      3.664    -23.915      0.000     -94.815     -80.450
yr_renovated     13.7791      3.005      4.585      0.000       7.889      19.669
sqft_living15    43.4332      2.942     14.763      0.000      37.666      49.200
sqft_lot15       -0.1876      0.078     -2.400      0.016      -0.341      -0.034
age             325.3285     64.495      5.044      0.000     198.910     451.747
2015             45.0769      2.896     15.563      0.000      39.400      50.754
2                 5.1018      3.034      1.681      0.093      -0.846      11.049
3                13.8074      2.806      4.920      0.000       8.307      19.308
4                20.5621      2.676      7.685      0.000      15.317      25.807
5                40.8991      3.103     13.179      0.000      34.816      46.982
6                49.0136      3.567     13.740      0.000      42.021      56.006
7                46.5068      3.565     13.047      0.000      39.519      53.494
8                50.8122      3.632     13.989      0.000      43.693      57.932
9                45.2744      3.655     12.388      0.000      38.111      52.438
10               50.4428      3.620     13.936      0.000      43.348      57.538
11               46.3142      3.748     12.358      0.000      38.968      53.660
12               48.2313      3.755     12.846      0.000      40.872      55.591
98002            16.7497      7.390      2.267      0.023       2.265      31.235
98003            14.6939      5.686      2.584      0.010       3.549      25.839
98004           336.1567      4.438     75.741      0.000     327.457     344.856
98005           155.9242      5.585     27.918      0.000     144.977     166.872
98006           153.4591      4.213     36.423      0.000     145.200     161.718
98007           143.9661      7.070     20.363      0.000     130.108     157.824
98008           140.3704      5.430     25.853      0.000     129.728     151.013
98010            54.3464      7.663      7.092      0.000      39.326      69.367
98011            85.1823      6.067     14.040   

# Feature Selection

In [207]:
lasso = Lasso(alpha=0.01, normalize=False)
lasso.fit(x_train, y_train)

y_train_pred = lasso.predict(x_train)
y_pred = lasso.predict(x_test)

In [208]:
train_rmse = metrics.mean_absolute_error(y_train, y_train_pred)
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('Training Error: '+ str(train_rmse) )
print('Testing Error: '+ str(test_rmse) )

Training Error: 78782.22627066355
Testing Error: 140419.01622902672


In [209]:
lasso_coef = pd.DataFrame(data=lasso.coef_).T
lasso_coef.columns = x_train.columns
lasso_coef = lasso_coef.T.sort_values(by=0).T

lasso_coef

,floors,bedrooms,sqft_basement,sqft_lot15,sqft_lot,98022,2,98023,98092,yr_renovated,...,98109,98112,age,98004,98102,98039,bathrooms,condition,view,grade
0,-43581.848933,-21443.788716,-87.632749,-0.187588,0.631029,3.165254,5.101769,6.408001,11.051442,13.779052,...,294.746319,312.065599,325.328489,336.156704,363.966229,466.765095,21396.346726,22893.826585,47156.655688,50879.812825


# Use Model to Forecast

In [210]:
# hold_df = hold_df.drop(columns=['Unnamed: 0', 'id', 'date', 'yr_built', 'zipcode',
#           'lat', 'long', 'sqft_living', 'sqft_above'])

In [211]:
y_hold = lasso.predict(x_hold)
y_hold

array([561620.93703266, 560182.76126881, 376700.44727703, ...,
       313011.74697455, 402365.49681327, 314743.86132152])

# Save Results

# Notes (TESTING)

In [212]:
# # Recreate original dataframe without any added columns
# TESTING = pd.read_csv('kc_house_data_train.csv')

# # Create features list for original dataframe
# TESTING_FEATURES = set_features(TESTING)

# # Run model on original dataframe
# set_model(TESTING, TESTING_FEATURES)

# Unused Functions

In [213]:
# # Function to create target variable
# def set_target(df):
#     target = df['price']
#     return target

# # Create target variable for dataframes
# train_target = set_target(train_df)
# test_target = set_target(test_df)

In [214]:
# # Function for creating grade dummy variables
# def set_grades(df):
#     dummies = pd.get_dummies(df['grade'], prefix='grade', drop_first=True)
#     df = pd.concat([df, dummies], axis=1)
#     return df

# # Create grade dummy variables for dataframes
# train_df = set_grades(train_df)
# holdout_df = set_grades(holdout_df)

In [215]:
# # Function to transform bedrooms outliers
# def bedrooms_trans(df):
#     filt = df['bedrooms'] < 10
#     df['bedrooms'] = np.where(filt, df['bedrooms'], 10)
#     return df

# # Transform bedrooms outliers in dataframes 
# train_df = bedrooms_trans(train_df)
# hold_df = bedrooms_trans(hold_df)

In [216]:
# # Function to transform bathrooms outliers
# def bathrooms_trans(df):
#     low_filt = df['bathrooms'] > .5
#     df['bathrooms'] = np.where(low_filt, df['bathrooms'], .5)
#     high_filt = df['bathrooms'] < 5
#     df['bathrooms'] = np.where(high_filt, df['bathrooms'], 5)
#     return df

# # Transform bedrooms outliers in dataframes 
# train_df = bathrooms_trans(train_df)
# hold_df = bathrooms_trans(hold_df)

In [217]:
# # Function to create regression modeln (smf method)
# def set_model(df, features):
#     formula = 'price~' + '+'.join([f'{ft}' for ft in features])
#     model = smf.ols(formula=formula, data=df).fit()
#     return model.summary()

# # Run model on train dataframe
# set_model(train_df, train_features)

In [218]:
# # Function to create age range dummy variables
# def set_ages(df):
#     year = df['yr_built']
#     # df['age_new'] = np.where(age==2015, 1, 0)
#     df['age_10'] = np.where((year>2004) & (year<2015), 1, 0)
#     df['age_20'] = np.where((year>1994) & (year<2005), 1, 0)
#     df['age_30'] = np.where((year>1984) & (year<1995), 1, 0)
#     df['age_40'] = np.where((year>1974) & (year<1985), 1, 0)
#     df['age_50'] = np.where((year>1964) & (year<1975), 1, 0)
#     df['age_60'] = np.where((year>1954) & (year<1965), 1, 0)
#     df['age_70'] = np.where((year>1944) & (year<1955), 1, 0)
#     df['age_80'] = np.where((year>1934) & (year<1945), 1, 0)
#     df['age_90'] = np.where((year>1924) & (year<1935), 1, 0)
#     df['age_100'] = np.where((year>1914) & (year<1925), 1, 0)
#     df['age_old'] = np.where((year<1915), 1, 0)
#     return df

# # Create age range dummy variables for dataframes
# # train_df = set_ages(train_df)
# # hold_df = set_ages(hold_df)

In [219]:
# # Function to create renovation age range dummy variables
# def set_ren_ages(df):
#     year = df['yr_built']
#     ren_yr = df['yr_renovated']
#     df['not_ren'] = np.where((year!=2015) & (ren_yr==0), 1, 0)
#     df['ren_5'] = np.where((ren_yr>2010), 1, 0)
#     df['ren_10'] = np.where((ren_yr>2005) & (ren_yr<2011), 1, 0)
#     df['ren_15'] = np.where((ren_yr>2000) & (ren_yr<2006), 1, 0)
#     df['ren_20'] = np.where((ren_yr>1995) & (ren_yr<2001), 1, 0)
#     df['ren_25'] = np.where((ren_yr>1990) & (ren_yr<1996), 1, 0)
#     df['ren_30'] = np.where((ren_yr>1985) & (ren_yr<1991), 1, 0)
#     # df['ren_old'] = np.where((ren_yr>0) & (ren_yr<1986), 1, 0)
#     return df

# # Create renovation age range dummy variables for dataframes
# # train_df = set_ren_ages(train_df)
# # hold_df = set_ren_ages(hold_df)

In [220]:
# # Transform grade variable
# df_test = train_df
# df_test['grade_log'] = df_test['grade'].apply(lambda x: math.log(x))

# df_test = train_df
# df_test['grade_exp'] = df_test['grade'].apply(lambda x: math.exp(x))

# df_test = train_df
# df_test['price_log'] = df_test['price'].apply(lambda x: math.log(x))


# test1 = df_test.groupby('grade_exp')['price'].mean()
# fig, ax = plt.subplots()
# ax.scatter(test1.index, test1.values)

In [221]:
# # Function to create features list
# def set_features(df):
#     ignore = ['Unnamed: 0', 'id', 'price', 'date', 'yr_built', 'zipcode',
#               'lat', 'long', 'sqft_living', 'sqft_above']
#     features = list(df.keys())
#     for feature in ignore:
#         if feature in features:
#             features.remove(feature)
#     return features

# # Create features list for dataframes
# features = set_features(train_df)

In [222]:
# # Function to create regression model
# def set_model(df, features):
#     model = sm.OLS(y_train, sm.add_constant(
#         df[[key for key in features]])).fit()
#     return model.summary()
# # Run model on training dataframe
# set_model(train_df, features)

In [223]:
# # Split into x dataframes and y series for train and test data
# Y = train_df['price']
# X = train_df.drop(['price'], axis=1)
# x_train, x_test, y_train, y_test = train_test_split(
#     X, Y, random_state=22,test_size=0.2)